In [1]:
import pandas as pd
import datetime
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import bokeh
import seaborn as sns
import pandas_profiling

In [5]:
#files = (glob.glob("data/data4/*.csv"))
files = (glob.glob("data/data2/*.csv"))

for file in files:
    print(file)

data/data2\tirol_sys_bat_1.csv
data/data2\tirol_sys_bat_10.csv
data/data2\tirol_sys_bat_11.csv
data/data2\tirol_sys_bat_12.csv
data/data2\tirol_sys_bat_13.csv
data/data2\tirol_sys_bat_14.csv
data/data2\tirol_sys_bat_15.csv
data/data2\tirol_sys_bat_18.csv
data/data2\tirol_sys_bat_2.csv
data/data2\tirol_sys_bat_20.csv
data/data2\tirol_sys_bat_21.csv
data/data2\tirol_sys_bat_23.csv
data/data2\tirol_sys_bat_24.csv
data/data2\tirol_sys_bat_25.csv
data/data2\tirol_sys_bat_26.csv
data/data2\tirol_sys_bat_27.csv
data/data2\tirol_sys_bat_3.csv
data/data2\tirol_sys_bat_4.csv
data/data2\tirol_sys_bat_5.csv
data/data2\tirol_sys_bat_6.csv
data/data2\tirol_sys_bat_7.csv
data/data2\tirol_sys_bat_8.csv
data/data2\tirol_sys_bat_9.csv


In [9]:

df = None
for file in files:
    df=pd.read_csv(file, error_bad_lines=False, low_memory=False, sep=";", decimal=",")
    #jahresstatistik
    print("Anlage aus CSV:"+file)
    
   

Anlage aus CSV:data/data2\tirol_sys_bat_1.csv
Anlage aus CSV:data/data2\tirol_sys_bat_10.csv
Anlage aus CSV:data/data2\tirol_sys_bat_11.csv
Anlage aus CSV:data/data2\tirol_sys_bat_12.csv
Anlage aus CSV:data/data2\tirol_sys_bat_13.csv
Anlage aus CSV:data/data2\tirol_sys_bat_14.csv
Anlage aus CSV:data/data2\tirol_sys_bat_15.csv
Anlage aus CSV:data/data2\tirol_sys_bat_18.csv
Anlage aus CSV:data/data2\tirol_sys_bat_2.csv
Anlage aus CSV:data/data2\tirol_sys_bat_20.csv
Anlage aus CSV:data/data2\tirol_sys_bat_21.csv
Anlage aus CSV:data/data2\tirol_sys_bat_23.csv
Anlage aus CSV:data/data2\tirol_sys_bat_24.csv
Anlage aus CSV:data/data2\tirol_sys_bat_25.csv
Anlage aus CSV:data/data2\tirol_sys_bat_26.csv
Anlage aus CSV:data/data2\tirol_sys_bat_27.csv
Anlage aus CSV:data/data2\tirol_sys_bat_3.csv
Anlage aus CSV:data/data2\tirol_sys_bat_4.csv
Anlage aus CSV:data/data2\tirol_sys_bat_5.csv
Anlage aus CSV:data/data2\tirol_sys_bat_6.csv
Anlage aus CSV:data/data2\tirol_sys_bat_7.csv
Anlage aus CSV:data/

In [10]:
df.dtypes

DateTime                             object
DevWork_97 [Energy_Ws]              float64
FromBattToConsumer_0 [Energy_Wh]    float64
FromGenToBatt_0 [Energy_Wh]         float64
FromGenToConsumer_0 [Energy_Wh]     float64
FromGenToGrid_0 [Energy_Wh]         float64
FromGridToConsumer_0 [Energy_Wh]    float64
FromGrid_0 [Energy_Wh]              float64
StateOfCharge_254 [Percent]         float64
ToConsumer_0 [Energy_Wh]            float64
dtype: object

In [13]:
#def jahresstatistik():
for file in files:
    
   #einlesen je nachdem, in welchem Format die Daten gespeichert sind: 
    try:
        df=pd.read_csv(file, error_bad_lines=False, low_memory=False, sep=";", decimal=",")
    except:
        df=pd.read_csv(file, error_bad_lines=False, low_memory=False, sep=",")
    
    
    
    # StateOfCharge wird nicht benötigt und somit gelöscht:
    try:
        del df["StateOfCharge_254 [Percent]"]
    except:
            pass
    #Datum formatieren:
    df['DateTime'] = pd.to_datetime(df['DateTime'], format="%Y-%m-%d", utc=True)
    df['DateTime'] = df['DateTime'].dt.tz_convert('Europe/Berlin')
    
    
    
    df.columns = ['DateTime', 'DevWork', 'FromBatToCons', 'FromGenToBat','FromGenToCons', 'FromGenToGrid','FromGridToCons','FromGrid', 'ToCons']
  
    
    
    
      #Da es sein kann, dass die Datentypen fälschlicherweise Objekte sind, wird das hier abgefangen und im Notfall korrigiert:
    try:
        df['FromGenToBat'] = df.FromGenToBat.astype(float)
    except:
        pass
    
    try:
        df['FromGenToCons'] = df.FromGenToCons.astype(float)
    except:
        pass
    
    try:
        df['FromGenToGrid'] = df.FromGenToGrid.astype(float)
    except:
        pass
    
    try:
        df['FromGridToCons'] = df.FromGridToCons.astype(float)
    except:
        pass
    
    try:
        df['FromGrid'] = df.FromGrid.astype(float)
    except:
        pass
    
    try:
        df['ToCons'] = df.ToCons.astype(float)
    except:
        pass
    
    try:
        del df['DevWork']
    except:
        pass
    
    #Erzeugt die Spalte FromGen dh. die Gesamterzeugungsmenge der Anlage
    df['FromGen'] = df['FromGenToBat'] + df['FromGenToCons'] + df['FromGenToGrid'] 
    
    
    
    #Falls 2021 nicht benötigt wird:
    df= df[(df['DateTime'].dt.year != 2021)]


    #Damit alle Werte nach Monaten ausgewertet werden können + Umwandlung in kWh

    df_aggr=df.groupby(pd.Grouper(key='DateTime',freq='M')).sum().reset_index()
    df_aggr['Datum'] = pd.to_datetime(df_aggr['DateTime']).dt.strftime('%Y-%m')
    df_aggr['Jahr'] = pd.to_datetime(df_aggr['DateTime']).dt.strftime('%Y')
    df_aggr['Monat'] = pd.to_datetime(df_aggr['DateTime']).dt.strftime('%m')

    #da die Features FromGenToBat  und FromBatToCons nicht in allen Anlagen vorhanden ist:
    try:
        df_aggr["FromBatToCons"] = df_aggr["FromBatToCons"]/1000
    except:
            pass
    try:
        df_aggr["FromGenToBat"] = df_aggr["FromGenToBat"]/1000
    except:
            pass

    df_aggr["FromGenToCons"] = df_aggr["FromGenToCons"]/1000
    df_aggr["FromGenToGrid"] = df_aggr["FromGenToGrid"]/1000
    df_aggr["FromGridToCons"] = df_aggr["FromGridToCons"]/1000
    df_aggr["FromGrid"] = df_aggr["FromGrid"]/1000
    df_aggr["ToCons"] = df_aggr["ToCons"]/1000
    df_aggr["FromGen"] = df_aggr["FromGen"]/1000
    del df_aggr['DateTime']
    #del df_aggr['DevWork']

    #num=file[-6]+file[-5]
    #num
    
    #Alle Felder aggregieren:
    verbr_jahr = df_aggr.groupby(df_aggr.Jahr ,as_index=True).sum().reset_index()
    print('Jahresmenge für Anlage: '+file) #num)
    print(verbr_jahr)
    print("")
    print("")
    

    #Speichert die Werte in das CSV:
  # ''' wird meist nicht beöntigt'''
'''
    speichern = None
    if speichern is None:
        verbr_jahr.to_csv('../data/data3/Mengen_pro_Jahr.csv', mode='a', index = False)
    else:
        with open('Mengen_pro_Jahr.csv','a') as fd:
            fd.write('Jahresmenge für Anlage: '+num)
            fd.write(verbr_jahr)

'''  
    
    #return verbr_jahr


Jahresmenge für Anlage: data/data2\tirol_sys_bat_1.csv
   Jahr  FromBatToCons  FromGenToBat  FromGenToCons  FromGenToGrid  \
0  2017     1576.89350    1725.83159      726.26776     1343.94125   
1  2018        2.63864       3.31657        5.93658        0.24624   
2  2019      968.71793    1089.35462      455.65662     1292.01724   
3  2020      148.75226     162.95513       60.52955      741.66348   

   FromGridToCons    FromGrid      ToCons     FromGen  
0      1862.04833  1919.12646  4091.69741  3796.04060  
1        67.47589    67.85700    76.05104     9.49939  
2       605.56381   625.61000  2029.74077  2837.02848  
3      1893.11285  1913.73983  2102.39641   965.14816  


Jahresmenge für Anlage: data/data2\tirol_sys_bat_10.csv
   Jahr  FromBatToCons  FromGenToBat  FromGenToCons  FromGenToGrid  \
0  2017     1117.78846    1265.53929     2264.96127     1005.34901   
1  2018     1077.63304    1206.28368     2203.72961      834.36028   
2  2019     1089.27925    1213.48454     2206.

Jahresmenge für Anlage: data/data2\tirol_sys_bat_24.csv
   Jahr  FromBatToCons  FromGenToBat  FromGenToCons  FromGenToGrid  \
0  2017     1632.10881    1852.17925      280.16726     5721.31785   
1  2018     1937.97336    2135.62043      354.92470     5028.04703   
2  2019     1833.10584    2054.96695      367.74922     5200.24179   
3  2020     1821.07963    2111.84152      363.83822     5533.57184   

   FromGridToCons   FromGrid      ToCons     FromGen  
0       576.78601  607.89904  2488.74302  7853.66436  
1       541.18385  561.54004  2834.09731  7518.59216  
2       576.05045  600.47700  2776.15763  7622.95796  
3       644.11727  661.46002  2828.78528  8009.25158  


Jahresmenge für Anlage: data/data2\tirol_sys_bat_25.csv
   Jahr  FromBatToCons  FromGenToBat  FromGenToCons  FromGenToGrid  \
0  2017     1131.22222    1255.82910     1622.41470     3639.59077   
1  2018     1370.92869    1520.51816     1507.72381     3771.41382   
2  2019     1302.40319    1425.97942     1401.5137

"\n    speichern = None\n    if speichern is None:\n        verbr_jahr.to_csv('../data/data3/Mengen_pro_Jahr.csv', mode='a', index = False)\n    else:\n        with open('Mengen_pro_Jahr.csv','a') as fd:\n            fd.write('Jahresmenge für Anlage: '+num)\n            fd.write(verbr_jahr)\n\n"